ベクトルを足さず、コサイン類似度のみで言葉を生成する。

In [1]:
from gensim.models import KeyedVectors
import json

# Word2Vecモデルのロード
model = KeyedVectors.load_word2vec_format('model_neo_nouns_only.bin', binary=True)

with open('word_pairs.json', 'r', encoding='utf-8') as f:
    word_pairs = json.load(f)

In [3]:

word = "天然水"
# model.similar_by_word(word)
lst = model.similar_by_key(word, topn=10)
lst

[('サントリー天然水', 0.7073501944541931),
 ('南アルプスの天然水', 0.6818466186523438),
 ('ナチュラルミネラルウォーター', 0.6715477705001831),
 ('ミネラルウォーター', 0.6250886917114258),
 ('アサヒおいしい水', 0.6134332418441772),
 ('おいしい水', 0.6005318760871887),
 ('缶チューハイ', 0.5496819019317627),
 ('名水', 0.5460743308067322),
 ('ザ・プレミアム・モルツ', 0.5439411997795105),
 ('カルピスウォーター', 0.5420613884925842)]

ベクトルを足したあとに生成した場合

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

vector_diff = [] # ベクトルの差のリスト
for word1, word2 in word_pairs:
    if word1 in model.key_to_index and word2 in model.key_to_index:
        vector_diff.append(model[word1] - model[word2])
        
v = np.concatenate(vector_diff)
kde = gaussian_kde(v)

pre_vec_diff = kde.resample(300).flatten()
vec_generated = model[word] + pre_vec_diff

model.similar_by_vector(vec_generated, topn=10)

[('天然水', 0.7850627303123474),
 ('南アルプスの天然水', 0.5530255436897278),
 ('サントリー天然水', 0.5445462465286255),
 ('ナチュラルミネラルウォーター', 0.4818604588508606),
 ('ミネラルウォーター', 0.46700742840766907),
 ('アサヒおいしい水', 0.45366722345352173),
 ('日本酒', 0.4358205199241638),
 ('おいしい水', 0.43400683999061584),
 ('千枚田', 0.42660924792289734),
 ('清酒', 0.41334670782089233)]

# 単語の頻度のランキングを出す。

In [1]:
import pandas as pd
import pickle
from tqdm import tqdm
from gensim.corpora import WikiCorpus
import MeCab
import ipadic
from collections import Counter
import re
import winsound

# ステップ1: データの解析
# wiki_dump_path = 'jawiki-latest-pages-articles.xml.bz2'
wiki_dump_path = 'bin/jawiki-latest-pages-articles.xml.bz2'
wiki_corpus = WikiCorpus(wiki_dump_path, dictionary={})

# ステップ3: 単語の頻度計算
word_counts = Counter()
mecab = MeCab.Tagger(ipadic.MECAB_ARGS)

for text in tqdm(wiki_corpus.get_texts()):
    res = "".join(text)
    parsed_text = mecab.parse(res)
    # print(parse_text)
    
    nouns = []
    for line in parsed_text.splitlines():
        if "名詞" in line and ("サ変接続" in line or "一般" in line):
            noun = line.split()[0]  # 名詞の単語部分を取得
            noun = re.sub(r'\b[a-zA-Z]\b','', noun)
            if noun != '':
                nouns.append(noun)
    word_counts.update(nouns)
    
    # print(word_counts)
most_common_words = word_counts.most_common()

# ランキングデータを表形式に変換
data = {"Word": [word for word, freq in most_common_words],
        "Frequency": [freq for word, freq in most_common_words]}
df = pd.DataFrame(data)
df
winsound.Beep(442, 3000)  # 1000 Hzの音を500ミリ秒間鳴らす

    # for ws in text:
        # word_counts.update(process_text(w))

0it [00:00, ?it/s]c:\Users\issei\AppData\Local\Programs\Python\Python311\Lib\site-packages\gensim\utils.py:1333: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected %s; aliasing chunkize to chunkize_serial" % entity)
934956it [59:30, 261.86it/s]


In [2]:
# pd.set_option('display.max_rows', 500)
# df.head(500)

df_a = df[df["Frequency"] >= 2000]
df_a

,Word,Frequency
0,年,3057925
1,日,1879591
2,月,1278188
3,者,1019071
4,人,982168
...,...,...
13924,イモ,2000
13925,ケロロ,2000
13926,悪戯,2000
13927,さそり,2000


In [4]:
df.to_pickle('bin/frec_jawiki-latest-pages-articles.pkl')

In [5]:
"昭和" in df_a["Word"].values

True